In [1]:
# import torch

# ckpt = torch.load("../../pretrained/G_1000.pth", map_location="cpu")  # или твой путь
# state = ckpt.get("model", ckpt)  # иногда чекпоинт лежит в ключе "model"

# dtypes = {k: v.dtype for k, v in state.items() if torch.is_tensor(v)}
# print(set(dtypes.values()))


In [2]:
import sys
import logging
from pathlib import Path

# Путь к корню репозитория, внутри которого лежит папка vosk_tts
repo_root = Path("/home/michael/Documents/ITMO/EDLM/phone-tts/vosk-tts")
sys.path.append(str(repo_root))

# теперь импортируем как пакет
from vosk_tts.model import Model
from vosk_tts.synth import Synth

logging.basicConfig(level=logging.INFO)

# Путь к распакованной TTS-модели Vosk (там должны лежать model.onnx, config.json, dictionary, etc.)
model_dir = Path("/home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi")  # подставь свой

# Загружаем исходный (не квантизованный) onnx
model = Model(model_path=model_dir, model_name=None, lang=None)

synth = Synth(model)


INFO:root:Loading model from /home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi


In [3]:

text = "Привет, это тест синтеза речи до квантизации модели."
out_wav = "pre_quant_test.wav"

synth.synth(
    text=text,
    oname=out_wav,
    speaker_id=2,
    noise_level=None,
    speech_rate=1.0,
    duration_noise_level=None,
    scale=None,
)

print("Готово, слушай:", out_wav)


INFO:root:Real-time factor: 0.17 (infer=0.75 sec, audio=4.39 sec)


Готово, слушай: pre_quant_test.wav


### Dinamic Q onnx

In [4]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

fp32_model_path = "/home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi/model.onnx"
int8_model_path = "/home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi_qd_int8/model.onnx"

model = onnx.load(fp32_model_path)

nodes_to_exclude = [
    node.name
    for node in model.graph.node
    if "time_mlp" in node.name
]

for n in nodes_to_exclude:
    print("  ", n)

extra_options = {
    "DefaultTensorType": onnx.TensorProto.FLOAT,
}

quantize_dynamic(
    model_input=fp32_model_path,
    model_output=int8_model_path,
    weight_type=QuantType.QInt8,
    op_types_to_quantize=["MatMul"],
    nodes_to_exclude=nodes_to_exclude,
    extra_options=extra_options,
)

print("quantized with exclusion, saved to", int8_model_path)



   /matcha/decoder/estimator/time_mlp/layer/layer.0/Constant
   /matcha/decoder/estimator/time_mlp/layer/layer.0/Gemm
   /matcha/decoder/estimator/time_mlp/layer/layer.1/Sigmoid
   /matcha/decoder/estimator/time_mlp/layer/layer.1/Mul
   /matcha/decoder/estimator/time_mlp/layer/layer.2/Gemm
   /matcha/decoder/estimator/time_mlp/layer/layer.0_1/Constant
   /matcha/decoder/estimator/time_mlp/layer/layer.0_1/Gemm
   /matcha/decoder/estimator/time_mlp/layer/layer.1_1/Sigmoid
   /matcha/decoder/estimator/time_mlp/layer/layer.1_1/Mul
   /matcha/decoder/estimator/time_mlp/layer/layer.2_1/Gemm
   /matcha/decoder/estimator/time_mlp/layer/layer.0_2/Constant
   /matcha/decoder/estimator/time_mlp/layer/layer.0_2/Gemm
   /matcha/decoder/estimator/time_mlp/layer/layer.1_2/Sigmoid
   /matcha/decoder/estimator/time_mlp/layer/layer.1_2/Mul
   /matcha/decoder/estimator/time_mlp/layer/layer.2_2/Gemm
   /matcha/decoder/estimator/time_mlp/layer/layer.0_3/Constant
   /matcha/decoder/estimator/time_mlp/layer/

INFO:root:Quantization parameters for tensor:"/matcha/dur_spk_emb/Gather_output_0" not specified
INFO:root:Quantization parameters for tensor:"/matcha/encoder/dp_encoder/encoder.0/adaLN_modulation/adaLN_modulation.1/Mul_output_0" not specified
INFO:root:Ignore MatMul due to non constant B: /[/matcha/encoder/dp_encoder/encoder.0/attn/MatMul]
INFO:root:Ignore MatMul due to non constant B: /[/matcha/encoder/dp_encoder/encoder.0/attn/MatMul_1]
INFO:root:Quantization parameters for tensor:"/matcha/encoder/dp_encoder/encoder.1/adaLN_modulation/adaLN_modulation.1/Mul_output_0" not specified
INFO:root:Ignore MatMul due to non constant B: /[/matcha/encoder/dp_encoder/encoder.1/attn/MatMul]
INFO:root:Ignore MatMul due to non constant B: /[/matcha/encoder/dp_encoder/encoder.1/attn/MatMul_1]
INFO:root:Quantization parameters for tensor:"/matcha/encoder/dp_encoder/encoder.2/adaLN_modulation/adaLN_modulation.1/Mul_output_0" not specified
INFO:root:Ignore MatMul due to non constant B: /[/matcha/encod

quantized with exclusion, saved to /home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi_qd_int8/model.onnx


In [5]:
qd_model_dir = Path("/home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi_qd_int8/") 

qd_model = Model(model_path=qd_model_dir, model_name=None, lang=None)

qd_synth = Synth(qd_model)

INFO:root:Loading model from /home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi_qd_int8


Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from /home/michael/Documents/ITMO/EDLM/phone-tts/vosk-model-tts-ru-0.9-multi_qd_int8/model.onnx failed:Node (/matcha/decoder/estimator/time_mlp/layer/layer.0/Gemm_MatMul_quant) Op (MatMulInteger) [ShapeInferenceError] Incompatible dimensions for matrix multiplication

In [ ]:
text = "Привет, это тест синтеза речи после квантизации модели."
out_wav = "post_quant_test.wav"

qd_synth.synth(
    text=text,
    oname=out_wav,
    speaker_id=2,
    noise_level=None,
    speech_rate=1.0,
    duration_noise_level=None,
    scale=None,
)

print(out_wav)

INFO:root:Real-time factor: 0.18 (infer=0.84 sec, audio=4.53 sec)


post_quant_test.wav


In [ ]:
import onnx

model = onnx.load(fp32_model_path)

for node in model.graph.node:
    if "time_mlp" in node.name:
        print(node.op_type, node.name)


Constant /matcha/decoder/estimator/time_mlp/layer/layer.0/Constant
Gemm /matcha/decoder/estimator/time_mlp/layer/layer.0/Gemm
Sigmoid /matcha/decoder/estimator/time_mlp/layer/layer.1/Sigmoid
Mul /matcha/decoder/estimator/time_mlp/layer/layer.1/Mul
Gemm /matcha/decoder/estimator/time_mlp/layer/layer.2/Gemm
Constant /matcha/decoder/estimator/time_mlp/layer/layer.0_1/Constant
Gemm /matcha/decoder/estimator/time_mlp/layer/layer.0_1/Gemm
Sigmoid /matcha/decoder/estimator/time_mlp/layer/layer.1_1/Sigmoid
Mul /matcha/decoder/estimator/time_mlp/layer/layer.1_1/Mul
Gemm /matcha/decoder/estimator/time_mlp/layer/layer.2_1/Gemm
Constant /matcha/decoder/estimator/time_mlp/layer/layer.0_2/Constant
Gemm /matcha/decoder/estimator/time_mlp/layer/layer.0_2/Gemm
Sigmoid /matcha/decoder/estimator/time_mlp/layer/layer.1_2/Sigmoid
Mul /matcha/decoder/estimator/time_mlp/layer/layer.1_2/Mul
Gemm /matcha/decoder/estimator/time_mlp/layer/layer.2_2/Gemm
Constant /matcha/decoder/estimator/time_mlp/layer/layer.0_3